Пример скрипта для генерации текста из произведений Ницше.
Требуется не менее 20 эпох, прежде чем сгенерированный текст
начнет звучать связно.
Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные
сети требуют довольно больших вычислительных затрат.

In [ ]:
# pip install --upgrade keras

**1) Загрузите текст из произведений Ницше ('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt').
Выведете следующее:
 А) длину всего корпуса;
 Б) количество предложений;
 В) сколько всего символов используется?**


In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils import get_file
import numpy as np
import random
import sys
import io
import nltk
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

600901/600901 [==============================] - 0s 1us/step


In [ ]:
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('Длина всего корпуса:', len(text))
print(text[:300])

Длина всего корпуса: 600893
preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to truth, ha


In [ ]:
sentences = nltk.sent_tokenize(text)

print('Количество предложений в тексте:', len(sentences))

print(sentences[:2])

Количество предложений в тексте: 2864
['preface\n\n\nsupposing that truth is a woman--what then?', 'is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman?']


In [ ]:
words = nltk.word_tokenize(text)
print('Количество слов в тексте:', len(words))
print(words[:10])

Количество слов в тексте: 117653
['preface', 'supposing', 'that', 'truth', 'is', 'a', 'woman', '--', 'what', 'then']


In [ ]:
character_counts = Counter(text)
sorted_characters = sorted(character_counts.items(), key=lambda x: x[1], reverse=True)

print('Кол-во различных символов в корпусе:', len(sorted_characters))

for char, count in sorted_characters:
    print(f'Символ "{char}" встречается {count} раз(а)')

Кол-во различных символов в корпусе: 57
Символ " " встречается 91208 раз(а)
Символ "e" встречается 59595 раз(а)
Символ "t" встречается 44450 раз(а)
Символ "i" встречается 37548 раз(а)
Символ "a" встречается 36796 раз(а)
Символ "o" встречается 36591 раз(а)
Символ "n" встречается 35598 раз(а)
Символ "s" встречается 34500 раз(а)
Символ "r" встречается 27224 раз(а)
Символ "h" встречается 26940 раз(а)
Символ "l" встречается 20336 раз(а)
Символ "d" встречается 16943 раз(а)
Символ "c" встречается 13655 раз(а)
Символ "u" встречается 13221 раз(а)
Символ "f" встречается 12222 раз(а)
Символ "m" встречается 11829 раз(а)
Символ "
" встречается 9934 раз(а)
Символ "p" встречается 9484 раз(а)
Символ "g" встречается 8843 раз(а)
Символ "y" встречается 8497 раз(а)
Символ "w" встречается 8329 раз(а)
Символ "," встречается 8201 раз(а)
Символ "b" встречается 5944 раз(а)
Символ "v" встречается 5273 раз(а)
Символ "-" встречается 3740 раз(а)
Символ "." встречается 2787 раз(а)
Символ "k" встречается 2147 раз(а)

**2) Сократите текст наполовину избыточными последовательностями символов maxlen**

In [ ]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 'ä': 53,
 'æ': 54,
 'é': 55,
 'ë': 56}

In [ ]:
maxlen = 40 #максимальная длину последовательности
step = 3 #количество символов между началом одной последовательности и началом следующей
sentences = [] #для хранения последовательностей
next_chars = [] #для хранения символов
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Кол-во последовательностей:', len(sentences))

print(sentences[:3])
print(next_chars[:3])
print('Векторизиция...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences): #итерация по всем последовательностям
    for t, char in enumerate(sentence): #итерация по всем символам в текущей последовательности
        x[i, t, char_indices[char]] = 1 #создание one-hot encoding для каждого символа в каждой последовательности
    y[i, char_indices[next_chars[i]]] = 1 #создание one-hot encoding для следующего символа после каждой последовательности

Кол-во последовательностей: 200285
['preface\n\n\nsupposing that truth is a woma', 'face\n\n\nsupposing that truth is a woman--', 'e\n\n\nsupposing that truth is a woman--wha']
['n', 'w', 't']
Векторизиция...


In [ ]:
# device = '/device:GPU:0'

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
x = tf.constant(x, dtype=tf.float32)
y = tf.constant(y, dtype=tf.float32)

 **Матрица x представляет one-hot encoding для каждого символа в каждой последовательности, вектор y представляет one-hot encoding для следующего символа после каждой последовательности. Эти данные могут быть использованы для обучения модели, предсказывающей следующий символ на основе предыдущих**

In [ ]:
x.shape, y.shape

(TensorShape([200285, 40, 57]), TensorShape([200285, 57]))

**Задание 3. Создайте модель LSTM для генерации текста
А) Напишите вспомогательную функцию для выборки индекса из массива вероятностей
Б) Напишите функцию, которая будет вызываться в конце каждой эпохи и печатать сгенерированный текст
В) Запустите модель на обучение
Имейте ввиду, что требуется не менее 20 эпох, прежде чем сгенерированный текст начнет звучать связно. Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные сети требуют довольно больших вычислительных затрат.
Г) Проверьте работу модели в онлайн режиме.**


In [ ]:
print('Build model...')
with tf.device('/device:GPU:0'):
  model = Sequential()
  model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #предназначен для обработки последовательностей данных
  model.add(Dense(len(chars), activation='softmax')) #каждый нейрон будет отвечать за предсказание вероятности появления определенного символа (поэтому колво нейронов=колву уникальных символов)

  optimizer = RMSprop(learning_rate=0.01) # отвечает за обновление параметров модели на основе функции потерь
  model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [ ]:
def sample(preds, temperature=1.0):
    """
    Принимает вероятности предсказания каждого символа и случайным образом выбирает следующий символ в соответствии с этими вероятностями и с учетом температуры.

    Parameters:
    - preds (list): Массив вероятностей для каждого символа.
    - temperature (float): Параметр температуры для регулирования разнообразия выбора.

    Returns:
    - int: Индекс выбранного символа.
    """
    with tf.device('/device:GPU:0'):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1) #мультиномиальное распределение для генерации случайного индекса на основе вероятностей
    return np.argmax(probas) #Возвращается индекс символа с наибольшей вероятностью


def on_epoch_end(epoch, _):
    """
    Функция вызывается в конце каждой эпохи обучения. Генерирует и печатает текст.

    Parameters:
    - epoch (int): Номер текущей эпохи.
    - _ (object): Параметр, который может быть проигнорирован. Используется в функции обратного вызова.

    Notes:
    - Функция использует генерацию текста с использованием обученной модели и различных значений разнообразия (diversity).
    - Для каждого значения diversity выводится сгенерированный текст.
    """
    with tf.device('/device:GPU:0'):
        print()
        print('----- Сгенерированный текст после эпохи: %d' % epoch)

        # Генерация случайного начального индекса для выбора начальной последовательности текста
        start_index = random.randint(0, len(text) - maxlen - 1)

        # Цикл по различным значениям diversity (если ближе к 0, то модель будет предсказывать наиболее вероятный символ с большей вероятностью. Это приведет к более предсказуемому тексту)
        # Если diversity увеличивается, то вероятность выбора менее вероятных символов также увеличивается. Это приводит к более разнообразному тексту
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            # Выбор начальной последовательности текста
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Генерация на основе начальной последовательности: "' + sentence + '"')
            sys.stdout.write(generated)

            # Генерация следующих 400 символов
            for i in range(400):
                # Подготовка входных данных для модели (one-hot encoding)
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                # Предсказание вероятностей следующего символа с использованием обученной модели
                preds = model.predict(x_pred, verbose=0)[0]
                # Получение следующего индекса символа с использованием функции sample
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                # Обновление последовательности текста для следующей итерации
                sentence = sentence[1:] + next_char

                # Вывод сгенерированного символа
                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

with tf.device('/device:GPU:0'):
    model.fit(x, y,
            batch_size=128,
            epochs=1,
    callbacks=[print_callback])

1563/1565 [============================>.] - ETA: 0s - loss: 1.6493
----- Сгенерированный текст после эпохи: 0
----- diversity: 0.2
----- Генерация на основе начальной последовательности: "deprived a father of his son and had him"
deprived a father of his son and had him is the self soul in the self so the self south the self and and sense in the self--as a man be the self to the self soul and the self-contrading to the self--and the self--and the sense of the self-conscience of the self-conception of the self and the sublement of the self to the self--as the self so case in the soul and an act the self and the self-delence to compression of the self-dention of t
----- diversity: 0.5
----- Генерация на основе начальной последовательности: "deprived a father of his son and had him"
deprived a father of his son and had him is the moral procession and the precide as to the controd to as case the content to the self-stronge a mank and even the to the science and consciences so and properat

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

with tf.device('/device:GPU:0'):
    model.fit(x, y,
            batch_size=128,
            epochs=20,
    callbacks=[print_callback])

Epoch 1/20
1560/1565 [============================>.] - ETA: 0s - loss: 2.0189
----- Сгенерированный текст после эпохи: 0
----- diversity: 0.2
----- Генерация на основе начальной последовательности: " poet, certainly: and poets
always know "
 poet, certainly: and poets
always know and such as an the spirit and inderstander the man and such distrust of the spiriture of the sperious and and mankind, the soul of the man instinct and and the soul the strugting to the strutte of the spirit and the man the senting of the spect it is as a soul to the procistule and the strugtions to the spirit and and and the stander the man and soul and the stope and instinct the stand and such a
----- diversity: 0.5
----- Генерация на основе начальной последовательности: " poet, certainly: and poets
always know "
 poet, certainly: and poets
always know have depentarily been in the prois, its
man instinct in the men an induct inself, that instancter for the reffire be the wand the men and and there is
perhen

In [ ]:
model.save('model_nizshe.h5')

**4) Создайте самостоятельно генерацию текста для РУССКОЯЗЫЧНОГО НАБОРА глав Wikibooks.
Полный текст Wikibooks содержит более 270000 глав на 12 языках https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/data**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3
db_path = '/content/drive/MyDrive/Colab Notebooks/wikibooks_data_ru.sqlite'

# Подключение к базе данных SQLite
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# Выполнение SQL-запроса для получения списка всех таблиц
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Извлечение названий таблиц
tables = cursor.fetchall()

# Вывод списка таблиц
for table in tables:
    print('Название таблицы:', table[0])

Название таблицы: wikibooks_table


In [ ]:
table_name = 'wikibooks_table'
cursor.execute(f"PRAGMA table_info({table_name});")
# Извлечение информации о столбцах
columns_info = cursor.fetchall()

# Вывод списка столбцов и их атрибутов
for column_info in columns_info:
    print(column_info)

(0, 'body_text', 'TEXT', 0, None, 0)


In [ ]:
cursor.execute("select body_text from wikibooks_table")
text = cursor.fetchall()
text[-5]

('Чики-пуки\n\nджин – 15 мл;\nликер из черной смородины – 15 мл;\nлимонный ликёр Лимончелло – 20 мл;\nчерный молотый перец – 1 грамм.\nЭтот крепкий коктейль наливают слоями в такой последовательности: джин, Лимончелло, черносмородинный ликер. \nЗатем поверхность стопки посыпают перцем.\nисточник: https://alcofan.com/prostye-koktejli-s-dzhinom-%e2%80%93-10-luchshix-receptov.html\nсм.также:\n\nhttp://www.alcorecept.ru/koktejli/koktejl-na-osnove-dzhina.html',)

In [ ]:
text = ' '.join([t[0].lower() for t in text])

# Выведем результат
print(text[:500])

рабочая станция;
сервер;
персональный компьютер. в википедии имеется статья по теме «свидетельство частного пилота»


гражданское пилотское свидетельство - разрешение на управление определенным видом воздушного судна. внутри этого свидетельства может быть много разных отметок (а может и не быть), которые или урезают или увеличивают количество разрешенных функций. самое понятное и известное массам это свидетельство частного пилота. его выдают после обучения на самолете или на вертолете. для кратк


In [ ]:
print('Длина всего корпуса:', len(text))

Длина всего корпуса: 55204420


In [ ]:
character_counts = Counter(text)
sorted_characters = sorted(character_counts.items(), key=lambda x: x[1], reverse=True)

print('Кол-во различных символов в корпусе:', len(sorted_characters))

for char, count in sorted_characters:
    print(f'Символ "{char}" встречается {count} раз(а)')

Выходные данные были обрезаны до нескольких последних строк (5000).
Символ "읔" встречается 1 раз(а)
Символ "읕" встречается 1 раз(а)
Символ "읖" встречается 1 раз(а)
Символ "읗" встречается 1 раз(а)
Символ "의" встречается 1 раз(а)
Символ "읙" встречается 1 раз(а)
Символ "읚" встречается 1 раз(а)
Символ "읛" встречается 1 раз(а)
Символ "읜" встречается 1 раз(а)
Символ "읝" встречается 1 раз(а)
Символ "읞" встречается 1 раз(а)
Символ "읟" встречается 1 раз(а)
Символ "읠" встречается 1 раз(а)
Символ "읡" встречается 1 раз(а)
Символ "읢" встречается 1 раз(а)
Символ "읣" встречается 1 раз(а)
Символ "읤" встречается 1 раз(а)
Символ "읥" встречается 1 раз(а)
Символ "읦" встречается 1 раз(а)
Символ "읧" встречается 1 раз(а)
Символ "읨" встречается 1 раз(а)
Символ "읩" встречается 1 раз(а)
Символ "읪" встречается 1 раз(а)
Символ "읫" встречается 1 раз(а)
Символ "읬" встречается 1 раз(а)
Символ "읭" встречается 1 раз(а)
Символ "읮" встречается 1 раз(а)
Символ "읯" встречается 1 раз(а)
Символ "읰" встречается 1 раз(а)
Симв

In [ ]:
import re
text = re.sub(r'[^а-яА-ЯёЁ\s.]', '', text)
text = re.sub(r'\s+', ' ', text)
text = text.strip()
print(text[:500])

рабочая станция сервер персональный компьютер. в википедии имеется статья по теме свидетельство частного пилота гражданское пилотское свидетельство разрешение на управление определенным видом воздушного судна. внутри этого свидетельства может быть много разных отметок а может и не быть которые или урезают или увеличивают количество разрешенных функций. самое понятное и известное массам это свидетельство частного пилота. его выдают после обучения на самолете или на вертолете. для краткости в разг


In [ ]:
character_counts = Counter(text)
sorted_characters = sorted(character_counts.items(), key=lambda x: x[1], reverse=True)

print('Кол-во различных символов в корпусе:', len(sorted_characters))

for char, count in sorted_characters:
    print(f'Символ "{char}" встречается {count} раз(а)')

Кол-во различных символов в корпусе: 35
Символ " " встречается 5293111 раз(а)
Символ "о" встречается 3487263 раз(а)
Символ "е" встречается 2918866 раз(а)
Символ "и" встречается 2855750 раз(а)
Символ "а" встречается 2634014 раз(а)
Символ "н" встречается 2333552 раз(а)
Символ "т" встречается 2229015 раз(а)
Символ "р" встречается 1816120 раз(а)
Символ "с" встречается 1812500 раз(а)
Символ "в" встречается 1504117 раз(а)
Символ "л" встречается 1342993 раз(а)
Символ "к" встречается 1174528 раз(а)
Символ "м" встречается 1087670 раз(а)
Символ "п" встречается 1076745 раз(а)
Символ "д" встречается 950417 раз(а)
Символ "у" встречается 745330 раз(а)
Символ "я" встречается 697533 раз(а)
Символ "ы" встречается 680939 раз(а)
Символ "з" встречается 586021 раз(а)
Символ "ь" встречается 553245 раз(а)
Символ "." встречается 545760 раз(а)
Символ "б" встречается 485050 раз(а)
Символ "г" встречается 478317 раз(а)
Символ "ч" встречается 465698 раз(а)
Символ "й" встречается 448942 раз(а)
Символ "х" встречаетс

In [ ]:
text = text[:500000]

In [ ]:
print(text[:200])

рабочая станция сервер персональный компьютер. в википедии имеется статья по теме свидетельство частного пилота гражданское пилотское свидетельство разрешение на управление определенным видом воздушно


In [ ]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
char_indices

{' ': 0,
 '.': 1,
 'а': 2,
 'б': 3,
 'в': 4,
 'г': 5,
 'д': 6,
 'е': 7,
 'ж': 8,
 'з': 9,
 'и': 10,
 'й': 11,
 'к': 12,
 'л': 13,
 'м': 14,
 'н': 15,
 'о': 16,
 'п': 17,
 'р': 18,
 'с': 19,
 'т': 20,
 'у': 21,
 'ф': 22,
 'х': 23,
 'ц': 24,
 'ч': 25,
 'ш': 26,
 'щ': 27,
 'ъ': 28,
 'ы': 29,
 'ь': 30,
 'э': 31,
 'ю': 32,
 'я': 33,
 'ё': 34}

In [ ]:
maxlen = 40 #максимальная длину последовательности
step = 2 #количество символов между началом одной последовательности и началом следующей
sentences = [] #для хранения последовательностей
next_chars = [] #для хранения символов
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Кол-во последовательностей:', len(sentences))

print(sentences[:3])
print(next_chars[:3])
print('Векторизиция...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences): #итерация по всем последовательностям
    for t, char in enumerate(sentence): #итерация по всем символам в текущей последовательности
        x[i, t, char_indices[char]] = 1 #создание one-hot encoding для каждого символа в каждой последовательности
    y[i, char_indices[next_chars[i]]] = 1 #создание one-hot encoding для следующего символа после каждой последовательности

Кол-во последовательностей: 249980
['рабочая станция сервер персональный комп', 'бочая станция сервер персональный компью', 'чая станция сервер персональный компьюте']
['ь', 'т', 'р']
Векторизиция...


In [ ]:
x.shape, y.shape

((249980, 40, 35), (249980, 35))

In [ ]:
input_shape = (40, 35)
output_dim = 35
batch_size = 64

In [ ]:
print('Build model...')
from keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten
from keras.layers import Reshape


with tf.device('/device:GPU:0'):
  model = Sequential()
  model.add(LSTM(1024, input_shape=input_shape))
  model.add(Dense(output_dim, activation='softmax'))

  # Компиляция модели с оптимизатором rmsprop
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Build model...


In [ ]:
def sample(preds, temperature=1.0):
    """
    Принимает вероятности предсказания каждого символа и случайным образом выбирает следующий символ в соответствии с этими вероятностями и с учетом температуры.

    Parameters:
    - preds (list): Массив вероятностей для каждого символа.
    - temperature (float): Параметр температуры для регулирования разнообразия выбора.

    Returns:
    - int: Индекс выбранного символа.
    """
    with tf.device('/device:GPU:0'):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1) #мультиномиальное распределение для генерации случайного индекса на основе вероятностей
    return np.argmax(probas) #Возвращается индекс символа с наибольшей вероятностью


def on_epoch_end(epoch, _):
    """
    Функция вызывается в конце каждой эпохи обучения. Генерирует и печатает текст.

    Parameters:
    - epoch (int): Номер текущей эпохи.
    - _ (object): Параметр, который может быть проигнорирован. Используется в функции обратного вызова.

    Notes:
    - Функция использует генерацию текста с использованием обученной модели и различных значений разнообразия (diversity).
    - Для каждого значения diversity выводится сгенерированный текст.
    """
    with tf.device('/device:GPU:0'):
        print()
        print('----- Сгенерированный текст после эпохи: %d' % epoch)

        # Генерация случайного начального индекса для выбора начальной последовательности текста
        start_index = random.randint(0, len(text) - maxlen - 1)

        # Цикл по различным значениям diversity (если ближе к 0, то модель будет предсказывать наиболее вероятный символ с большей вероятностью. Это приведет к более предсказуемому тексту)
        # Если diversity увеличивается, то вероятность выбора менее вероятных символов также увеличивается. Это приводит к более разнообразному тексту
        for diversity in [0.2]:
            print('----- diversity:', diversity)

            generated = ''
            # Выбор начальной последовательности текста
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Генерация на основе начальной последовательности: "' + sentence + '"')
            sys.stdout.write(generated)

            # Генерация следующих 400 символов
            for i in range(100):
                # Подготовка входных данных для модели (one-hot encoding)
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                # Предсказание вероятностей следующего символа с использованием обученной модели
                preds = model.predict(x_pred, verbose=0)[0]
                # Получение следующего индекса символа с использованием функции sample
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                # Обновление последовательности текста для следующей итерации
                sentence = sentence[1:] + next_char

                # Вывод сгенерированного символа
                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

with tf.device('/device:GPU:0'):
    model.fit(x, y,
            batch_size=64,
            epochs=15,
    callbacks=[print_callback])

Epoch 1/15
3906/3906 [==============================] - ETA: 0s - loss: 2.1591 - accuracy: 0.3618
----- Сгенерированный текст после эпохи: 0
----- diversity: 0.2
----- Генерация на основе начальной последовательности: "ия поиск нового функциональностоимостный"
ия поиск нового функциональностоимостный подома на продукта подом по всем на по страниции по статия последней соменные постоянных постоянных
3906/3906 [==============================] - 110s 27ms/step - loss: 2.1591 - accuracy: 0.3618
Epoch 2/15
3905/3906 [============================>.] - ETA: 0s - loss: 1.5758 - accuracy: 0.5237
----- Сгенерированный текст после эпохи: 1
----- diversity: 0.2
----- Генерация на основе начальной последовательности: "ное в инессиве.при прибавлении окончания"
ное в инессиве.при прибавлении окончания продаж по предприятия продукции и области продукции и области продукции и области продукции и облас
3906/3906 [==============================] - 111s 28ms/step - loss: 1.5758 - accuracy: 0.5237
Epoch 3/

In [ ]:
model.save('model_ru.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('model_ru.h5')

In [ ]:
maxlen = 40 #максимальная длину последовательности
step = 2

In [ ]:
def generate_text(model, start_text, char_indices, indices_char, maxlen, diversity, length=50):
    """
    Генерация текста с использованием обученной модели.

    Parameters:
    - model (tf.keras.Model): Обученная модель.
    - start_text (str): Начальный текст для генерации.
    - char_indices (dict): Словарь символов в индексы.
    - indices_char (dict): Словарь индексов в символы.
    - maxlen (int): Длина подпоследовательности.
    - diversity (float): Параметр температуры для регулирования разнообразия выбора.
    - length (int): Длина генерируемого текста.

    Returns:
    - str: Сгенерированный текст.
    """
    generated = start_text
    sentence = start_text

    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

    return generated

# Пример использования:
start_text = True
maxlen = 40
while start_text != 'стоп':
  start_text = input('Введите текст для основы генерации: ')
  generated_text = generate_text(model, start_text, char_indices, indices_char, maxlen, diversity=0.2)
  print('Сгенерированный текст:\n', generated_text)


Введите текст для основы генерации: рыбы плавают
Сгенерированный текст:
 рыбы плаваютря.рюх ту  . лбр  врк л у т.в  рэлкнниу   г влуиюх
Введите текст для основы генерации: и. вебер писал что в ссср властвует дикт
Сгенерированный текст:
 и. вебер писал что в ссср властвует диктетический раздел по материальным заданиямправить м


KeyboardInterrupt: ignored